This script is convert raw residential permit data to import ready PSRC data 

In [1]:
import pandas as pd 
import numpy as np
import os
os.getcwd()

'T:\\2018March\\Angela'

In [5]:
temp = pd.read_csv('template.csv')
raw_data = pd.read_csv('033UNINCORPORATEDKING.csv')
PIN_info = pd.read_csv('033UNINCORPORATEDKING3.csv')

In [6]:
test1 = raw_data 

In [7]:
my_permit_id = 'PERMIT_NUMBER' #the column name for permit id from raw data
my_address = 'ADDRESS'

# Sort 

Creat Sort ID by following previous format. The format could be different from city to city.  

In [8]:
test1['SORT'] = ''
for i in range(len(test1)): 
    p = test1[my_permit_id][i]
    p1 = '20' + p[4:6] + p[0:4] + '-' + p[-4:]
    test1['SORT'][i] = p1

c:\anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
test1['SORT'].head()

0    2014ADDC-0625
1    2015ADDC-0124
2    2015ADDC-0196
3    2015ADDC-0218
4    2015ADDC-0419
Name: SORT, dtype: object

# Site Address

split full address into ['HOUSENO', 'PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP'] columns

In [10]:
test1[my_address].head()

0                  10722 244TH AVE NE 98053
1                  32437 190TH AVE SE 98092
2                  14015 240TH AVE SE 98027
3    39711 248TH AVE SE, ENUMCLAW, WA 98022
4                  16704 162ND AVE NE 98072
Name: ADDRESS, dtype: object

In [13]:
col1 = ['HOUSENO', 'PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP']
for c in col1:
    test1[c] = ''

In [14]:
# Roughly split full adress into columns, without QA/QC
for i in range(len(test1)):
    test1[my_address][i] = test1[my_address][i].replace(',', ' ')
    address = test1[my_address][i].split()
    if len(address) <= 9:
        for j in range(len(address)): 
            test1[col1[j]][i] = address[j]
    else: 
        print address 
        #for j in range(4): #why 4? already investigated (the default is 6)
            #test1[col1[j]][i] = address[j+2]

c:\anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
city_list = ['Auburn', 'AUBURN', 'RENTON', 'ENUMCLAW', 'Enumclaw', 'BEND', 'CARNATION', 
            'DUVALL', 'ISSAQUAH', 'Issaquah', 'KENT', 'King', 'PRESTON', 'RAVENSDALE',
            'REDMOND', 'Renton', 'SAMMAMISH', 'SNOQUALMIE', 'VASHON', 'WOODINVILLE',
            ]

list_prefix = ['S', 'SE', 'SW', 'W', 'NW', 'N','NE','E']
list_suffix = ['S', 'SE', 'SW', 'W', 'NW', 'N','NE','E']

street_type = ['AVE', 'BLVD', 'CIR','CIRCLE','CROSSING', 'COVE', 'CT', 'DR', 'HWY', 
               'LN', 'LINE', 'LOOP', 'PL', 'RD', 'ROAD','Road','SHORE',
               'ST', 'TRAIL', 'TRL',  'WAY']

In [16]:
# Fix the prefix 
for i in range (len(test1)): 
    if test1['PREFIX'][i] not in list_prefix: 
        test1.loc[i:i,['PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
    else:
        continue 

In [18]:
backup_test1 = test1 # just in case, if I lost all data later 

In [17]:
# use ['SUFFIX'] as the base to inspect the row # data, QA/QC street name and unit building name.
for i in range(len(test1)): 
    #print test1['SUFFIX'][i]
    if len(test1['ADDRESS'][i].split()) > 4: 
        #print i 
        if str(test1['SUFFIX'][i]) in street_type: 
            test1['STRNAME'][i] = test1['STRNAME'][i] + ' ' + test1['STRTYPE'][i] 
            test1.loc[i:i,['STRTYPE', 'SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['STRTYPE', 'SUFFIX', 'UNIT_BLD']].shift(-1,axis=1)
        else: 
            if str(test1['SUFFIX'][i]) in list_suffix: # suffix is suffix
                continue
            if str(test1['SUFFIX'][i]) in city_list: # city name in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
            if str(test1['SUFFIX'][i]) in ['WA']: # state name in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(2,axis=1)
            if str(test1['SUFFIX'][i])[0] == 9: # zipcode in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(2,axis=1)
            else: #may be street name in suffix ? or nothing in suffix 
                #print test1['ADDRESS'][i]
                continue
            #test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
    else: 
        continue 

c:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
# QA/QC surffix
for i in range (len(test1)): 
    if test1['SUFFIX'][i] not in list_suffix: 
        test1.loc[i:i,['SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP']].shift(1,axis=1)
    else:
        continue 

In [20]:
# QA/QC unit building number 
for i in range (len(test1)): 
    if test1['UNIT_BLD'][i] not in ['1', '']: 
        test1.loc[i:i,['UNIT_BLD', 'CITY', 'STATE', 'ZIP']] = test1.loc[i:i,['UNIT_BLD', 'CITY', 'STATE', 'ZIP']].shift(1,axis=1)
    else:
        continue 

In [21]:
for i in range (len(test1)): 
    if test1['CITY'][i] not in city_list: 
        test1.loc[i:i,['CITY', 'STATE', 'ZIP']] = test1.loc[i:i,[ 'CITY', 'STATE', 'ZIP']].shift(1,axis=1)
    else:
        continue 

In [22]:
for i in range (len(test1)): 
    if test1['STATE'][i] not in ['WA']: 
        test1.loc[i:i,['STATE', 'ZIP']] = test1.loc[i:i,[ 'STATE', 'ZIP']].shift(1,axis=1)
    else:
        continue 

In [23]:
np.unique(test1['ZIP'])

c:\anaconda\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '', '***',
       '98001', '98003', '98010', '98011', '98014', '98019', '98022',
       '98024', '98027', '98029', '98032', '98038', '98042', '98045',
       '98051', '98052', '98053', '98058', '98059', '98065', '98070',
       '98072', '98074', '98077', '98092', '98106', '98146', '98168',
       '98178', 'CITY', 'DESIRE', 'DIAMOND', 'LAKE', 'MAPLE', 'NORTH',
       'SEATTLE', 'VALLEY'], dtype=object)

In [24]:
test1 = test1.replace(np.nan, '', regex=True)

# Type 

TYPE IS VERY DIFFICULT TO CODE, will process this via excel manually in the end. 

# NOTES, 2, 3, 4, 5

In [26]:
test2 = test1

In [27]:
for c in ['NOTES', 'NOTES2', 'NOTES3', 'NOTES4', 'NOTES5']:
    test2[c] = ''

In [28]:
test2.columns

Index([u'PERMIT_NUMBER', u'Group', u'Type', u'Sub-Type', u'Category',
       u'PROJECT_NAME', u'DETAIL_DESCRIPTION', u'APPLICATION_DATE',
       u'ISSUED_DATE', u'STATUS', u' JOB_VALUE ', u'PARCEL',
       u'SITE ADDRESS_LOCATION', u'ADDRESS', u'APPLICANT_NAME_ADDR',
       u'OWNER_NAME_ADDR', u'NBR_OF_DWELLING_UNITS', u'REVISION',
       u'COMMUNITY_SERVICE_AREA', u'COUNCEL_DISTRICT', u'PERMIT_TYPE',
       u'Month', u'Quarter', u'Year', u'Issuing Agency', u'FOR PERIOD ',
       u'SORT', u'HOUSENO', u'PREFIX', u'STRNAME', u'STRTYPE', u'SUFFIX',
       u'UNIT_BLD', u'CITY', u'STATE', u'ZIP', u'NOTES', u'NOTES2', u'NOTES3',
       u'NOTES4', u'NOTES5'],
      dtype='object')

In [29]:
test2['NOTES'] = test2['PROJECT_NAME']
#test2['NOTES2'] = test2['Lot Area/Size']
test2['NOTES3'] = test2['Type']
test2['NOTES5'] = test2['STATUS']
test2['NOTES6'] = test2['Type'] # just reserve some info from raw data
test2['NOTES7'] = test2['Sub-Type'] # reserve some info from raw data 
for i in range(len(test2)):
    work = test2['DETAIL_DESCRIPTION'][i]
    test2['NOTES4'][i] = work[:50]


c:\anaconda\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Extra information to carry over

The first two digits indicate the project year. The second two digits indicate the county in which the permit is located by abbreviated FIPS county code. The next five digits (included since 2000) indicate the jurisdiction of permit issue by FIPS place code. The last five digits serve as an automatic counter.

In [30]:
test3 = test2

In [31]:
test3['PERMITNO'] = test3['PERMIT_NUMBER']

In [32]:
test3['PSRCIDN'] = '' # no data 
test3['PIN'] = '' # no data 
test3['TYPE'] = ''
test3['PS'] = ''
test3['MULTIREC'] = ''
test3['LANDUSE'] = ''
test3['CONDO'] = ''
test3['ZONING'] = ''
test3['LOTSIZE'] = '' 
test3['PIN_PARENT'] = ''
test3['BLDGS'] = 1
test3['FINALED'] = ''
test3['ISSUED'] = test3['ISSUED_DATE']
test3['STATUS'] = test3['STATUS']
test3['VALUE'] = test3[' JOB_VALUE ']
test3['UNITS'] = test3['NBR_OF_DWELLING_UNITS']

In [33]:
test3['SORT'] = ''
for i in range(len(test3)): 
    p = test3[my_permit_id][i]
    p1 = '20' + p[4:6] + p[0:4] + '-' + p[-4:]
    test3['SORT'][i] = p1

c:\anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Template 

In [35]:
final_col = list(temp.columns.values) + ['CITY', 'STATE'] 

In [36]:
test3.columns

Index([u'PERMIT_NUMBER', u'Group', u'Type', u'Sub-Type', u'Category',
       u'PROJECT_NAME', u'DETAIL_DESCRIPTION', u'APPLICATION_DATE',
       u'ISSUED_DATE', u'STATUS', u' JOB_VALUE ', u'PARCEL',
       u'SITE ADDRESS_LOCATION', u'ADDRESS', u'APPLICANT_NAME_ADDR',
       u'OWNER_NAME_ADDR', u'NBR_OF_DWELLING_UNITS', u'REVISION',
       u'COMMUNITY_SERVICE_AREA', u'COUNCEL_DISTRICT', u'PERMIT_TYPE',
       u'Month', u'Quarter', u'Year', u'Issuing Agency', u'FOR PERIOD ',
       u'SORT', u'HOUSENO', u'PREFIX', u'STRNAME', u'STRTYPE', u'SUFFIX',
       u'UNIT_BLD', u'CITY', u'STATE', u'ZIP', u'NOTES', u'NOTES2', u'NOTES3',
       u'NOTES4', u'NOTES5', u'NOTES6', u'NOTES7', u'PERMITNO', u'PSRCIDN',
       u'PIN', u'TYPE', u'PS', u'MULTIREC', u'LANDUSE', u'CONDO', u'ZONING',
       u'LOTSIZE', u'PIN_PARENT', u'BLDGS', u'FINALED', u'ISSUED', u'VALUE',
       u'UNITS'],
      dtype='object')

In [37]:
df = test3[final_col]

# PIN

In [41]:
df_pin = PIN_info[['PERMITNO', 'PIN']]

In [42]:
left=df
right=df_pin
df = pd.merge(left, right, how='left', on='PERMITNO')

In [43]:
df['PIN_y'] = df['PIN_y'].astype(str)

In [67]:
df.to_csv('final_unincorporated_king_withPIN.csv')